In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

We try with the dataset with the cyclic time periods, and a shortened one to see if that changes the hyper-paramaters

In [20]:
df_cyclic = pd.read_pickle('../data/fe_temp_cyclic_data.pkl')
df_cyclic_short = df_cyclic[df_cyclic.DateTime > '2022-09'].copy()

In [ ]:
split_date = '2023-10-01' 

train_df = df_cyclic_short[df_cyclic_short['DateTime'] < split_date]
test_df = df_cyclic_short[df_cyclic_short['DateTime'] >= split_date]

# We need to remove the object type columns
X_train = train_df.drop(columns=['demand','DateTime','date','time'])
y_train = train_df['demand']

X_test = test_df.drop(columns=['demand','DateTime','date','time'])
y_test = test_df['demand']


In [ ]:
def objective(trial):

    # here is the paramaters ranges we're looking within
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    subsample = trial.suggest_float('subsample', 0.7, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.7, 1.0)
    gamma = trial.suggest_float('gamma', 0, 0.3)

    xgb_model = XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        random_state=5
    )
    
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print(f"Best parameters found: {best_params}")

# with full time window

[I 2024-09-11 10:40:34,266] Trial 44 finished with value: 0.11240548642264858 and parameters: {'n_estimators': 765, 'learning_rate': 0.09598085705977137, 'max_depth': 3, 'subsample': 0.740170271525394, 'colsample_bytree': 0.8993574574710597, 'gamma': 0.29477848229077347}. Best is trial 44 with value: 0.11240548642264858.


Best parameters found: {'n_estimators': 765, 'learning_rate': 0.09598085705977137, 'max_depth': 3, 'subsample': 0.740170271525394, 'colsample_bytree': 0.8993574574710597, 'gamma': 0.29477848229077347}

# with short time window

[I 2024-09-11 11:02:35,078] Trial 83 finished with value: 0.11306882108061495 and parameters: {'n_estimators': 805, 'learning_rate': 0.0572210313213236, 'max_depth': 3, 'subsample': 0.7193063172814127, 'colsample_bytree': 0.9945420576187365, 'gamma': 0.20977825247196802}. Best is trial 83 with value: 0.11306882108061495.
